In [1]:
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.autograd import Variable

import torch.backends.cudnn as cudnn
cudnn.benchmark = True

In [2]:
hidden_size = 64
num_layers = 1
learning_rate = 0.01
EPOCH = 600

In [3]:
class LSTMRG(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=1, num_layers=2):
        super(LSTMRG, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers)
        self.reg = nn.Linear(hidden_size, output_size)  # 拼接全连接层

    def forward(self, x):
        x, _ = self.rnn(x)
        s, b, h = x.shape  # (seq, batch, hidden_size)
        x = x.view(s * b, h)
        x = self.reg(x)
        x = x.view(s, b, -1)
        return x

In [4]:
pred_day = 28
metro_file = pd.read_csv('../../Metro_testA/testA_record_2019-01-'+str(pred_day).zfill(2)+'.csv')
metro_file.time = pd.to_datetime(metro_file.time)
start_time = '2019-01-'+str(pred_day)+' 00:00:00'
start_time = pd.to_datetime(start_time)
end_time = '2019-01-'+str(pred_day)+' 00:10:00'
end_time = pd.to_datetime(end_time)
tenmin_delta = end_time - start_time
metro_dfs = []
for k in range(81):
    metro_dfs.append(pd.DataFrame(np.zeros((144, 11))))
for j in range(144):
    metro_tmp = metro_file[(metro_file.time>=start_time) & (metro_file.time<end_time)].groupby(['stationID', 'status', 'payType']).count()
    for k in range(81):
        try:
            metro_dfs[k].loc[j, 0] = metro_tmp.loc[k, 1].time.sum()
        except:
            metro_dfs[k].loc[j, 0] = 0
        try:
            metro_dfs[k].loc[j, 1] = metro_tmp.loc[k, 1, 0].time
        except:
            metro_dfs[k].loc[j, 1] = 0
        try:
            metro_dfs[k].loc[j, 2] = metro_tmp.loc[k, 1, 1].time
        except:
            metro_dfs[k].loc[j, 2] = 0
        try:
            metro_dfs[k].loc[j, 3] = metro_tmp.loc[k, 1, 2].time
        except:
            metro_dfs[k].loc[j, 3] = 0
        try:
            metro_dfs[k].loc[j, 4] = metro_tmp.loc[k, 1, 3].time
        except:
            metro_dfs[k].loc[j, 4] = 0
        try:
            metro_dfs[k].loc[j, 5] = metro_tmp.loc[k, 0].time.sum()
        except:
            metro_dfs[k].loc[j, 5] = 0
        try:
            metro_dfs[k].loc[j, 6] = metro_tmp.loc[k, 0, 0].time
        except:
            metro_dfs[k].loc[j, 6] = 0
        try:
            metro_dfs[k].loc[j, 7] = metro_tmp.loc[k, 0, 1].time
        except:
            metro_dfs[k].loc[j, 7] = 0
        try:
            metro_dfs[k].loc[j, 8] = metro_tmp.loc[k, 0, 2].time
        except:
            metro_dfs[k].loc[j, 8] = 0
        try:
            metro_dfs[k].loc[j, 9] = metro_tmp.loc[k, 0, 3].time
        except:
            metro_dfs[k].loc[j, 9] = 0
        if start_time.weekday()<=4:
            metro_dfs[k].loc[j, 10] = 0
        else:
            metro_dfs[k].loc[j, 10] = 1
    start_time = end_time
    end_time += tenmin_delta

In [5]:
for i in range(81):
    metro_dfs[i].rename(columns={0:'in_num', 1:'in_paytype_0', 2:'in_paytype_1', 3:'in_paytype_2', 4:'in_paytype_3', 
                                 5:'out_num', 6:'out_paytype_0', 7:'out_paytype_1', 8:'out_paytype_2', 9:'out_paytype_3',
                                 10:'weekend'}, inplace=True)

In [6]:
result = np.zeros((11664, 2))
all_min_max = np.load('all_min_max.npy').item()
eval_feature = np.zeros((81, 2, 144, 1, 8))
for istation in range(81):
    if istation != 54:
        istation_min_max = all_min_max['station-'+str(istation).zfill(2)]
        metro_dfs[istation].iloc[:, [0,5,1,2,3,4,6,7,8,9]] = (metro_dfs[istation].iloc[:, [0,5,1,2,3,4,6,7,8,9]] - np.array([istation_min_max['in']['in_num_min'],
                                                       istation_min_max['in']['out_num_min'],
                                                       istation_min_max['in']['in_paytype_0_min'],
                                                       istation_min_max['in']['in_paytype_1_min'],
                                                       istation_min_max['in']['in_paytype_2_min'],
                                                       istation_min_max['in']['in_paytype_3_min'],
                                                       istation_min_max['out']['out_paytype_0_min'],
                                                       istation_min_max['out']['out_paytype_1_min'],
                                                       istation_min_max['out']['out_paytype_2_min'],
                                                       istation_min_max['out']['out_paytype_3_min'],
                                                      ]))/(np.array([istation_min_max['in']['in_num_max'],
                                                                    istation_min_max['in']['out_num_max'],
                                                                    istation_min_max['in']['in_paytype_0_max'],
                                                                    istation_min_max['in']['in_paytype_1_max'],
                                                                    istation_min_max['in']['in_paytype_2_max'],
                                                                    istation_min_max['in']['in_paytype_3_max'],
                                                                    istation_min_max['out']['out_paytype_0_max'],
                                                                    istation_min_max['out']['out_paytype_1_max'],
                                                                    istation_min_max['out']['out_paytype_2_max'],
                                                                    istation_min_max['out']['out_paytype_3_max'],
                                     ])-np.array([istation_min_max['in']['in_num_min'],
                                                  istation_min_max['in']['out_num_min'],
                                                  istation_min_max['in']['in_paytype_0_min'],
                                                  istation_min_max['in']['in_paytype_1_min'],
                                                  istation_min_max['in']['in_paytype_2_min'],
                                                  istation_min_max['in']['in_paytype_3_min'],
                                                  istation_min_max['out']['out_paytype_0_min'],
                                                  istation_min_max['out']['out_paytype_1_min'],
                                                  istation_min_max['out']['out_paytype_2_min'],
                                                  istation_min_max['out']['out_paytype_3_min'],
                                                 ]))
        for jinout in range(2):
            if jinout == 0:
                one_feature = metro_dfs[istation].iloc[:,[0,5,1,2,3,4,10]]
            else:
                one_feature = metro_dfs[istation].iloc[:,[0,5,6,7,8,9,10]]
            if pd.to_datetime('2019-01-'+str(pred_day+1).zfill(2)).weekday()<=4:
                weekend = 0 * np.ones((144, 1))
            else:
                weekend = 1 * np.ones((144, 1))
            one_feature = np.hstack((one_feature, weekend))
            one_feature = np.expand_dims(one_feature, axis=1)
            eval_feature[istation, jinout, :, :, :] = one_feature

In [7]:
result = np.zeros((11664, 2))
for istation in range(81):
    for jinout in range(2):
        net = LSTMRG(input_size = eval_feature.shape[-1], hidden_size = hidden_size, output_size=1, 
                     num_layers=num_layers)
        net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
        net.cuda()
        net.load_state_dict(torch.load('model/station-'+str(istation).zfill(2)+'-'+str(jinout)+'.pth'))
        net.eval()
        
        one_feature = torch.from_numpy(eval_feature[istation][jinout])
        one_feature = one_feature.float()
        one_feature = Variable(one_feature)
        one_feature.cuda()
        one_pred = net(one_feature)
        one_pred = one_pred.cpu().detach().numpy()
        
        if jinout == 0:
            one_min = all_min_max['station-'+str(istation).zfill(2)]['in']['in_num_min']
            one_max = all_min_max['station-'+str(istation).zfill(2)]['in']['in_num_max']
        else:
            one_min = all_min_max['station-'+str(istation).zfill(2)]['out']['in_num_min']
            one_max = all_min_max['station-'+str(istation).zfill(2)]['out']['in_num_max']
        one_pred = one_pred[:, 0, 0] * (one_max - one_min) + one_min
        one_pred = pd.DataFrame(one_pred)
        one_pred[one_pred.iloc[:,0]<0] = 0
        one_pred = np.array(one_pred).round().reshape(-1)
        result[istation*144:(istation+1)*144, jinout] = one_pred

In [8]:
result = pd.DataFrame(result)
result.to_csv('pred.csv', index=False)